In [15]:
import os
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd

In [16]:
data1=pd.read_csv('../Data/startup_data/2023-02-27-yc-companies.csv')
data2=pd.read_csv('../Data/startup_data/2023-07-13-yc-companies.csv')

data = pd.concat([data1, data2], ignore_index=True)
data.shape

(12538, 16)

In [17]:
import pandas as pd
def combine_rows(group):
    return {
        'company_id': ' \n '.join(group['company_id'].astype(str).unique()),
        'short_description': ' \n '.join(group['short_description'].dropna().unique()),
        'long_description': ' \n '.join(group['long_description'].dropna().unique()),
        'batch': ' , '.join(group['batch'].astype(str).dropna().unique()),
        'status': ' , '.join(group['status'].astype(str).dropna().unique()),
        'tags': ' , '.join(group['tags'].dropna().unique()),
        'location': ' , '.join(group['location'].dropna().unique()),
        'country': ' , '.join(group['country'].dropna().unique()),
        'year_founded': ' , '.join(group['year_founded'].astype(str).dropna().unique()),
        'num_founders': ' , '.join(group['num_founders'].astype(str).dropna().unique()),
        'founders_names': ' , '.join(group['founders_names'].dropna().unique()),
        'team_size': ' , '.join(group['team_size'].astype(str).dropna().unique()),
        'website': ' , '.join(group['website'].dropna().unique()),
        'cb_url': ' , '.join(group['cb_url'].dropna().unique()),
        'linkedin_url': ' , '.join(group['linkedin_url'].dropna().unique())
    }



result_df = data.groupby('company_name').apply(combine_rows).apply(pd.Series)
result_df.reset_index(inplace=True)

/var/folders/p5/2hshz6dj0fxcd6lwkfj4c4vm0000gp/T/ipykernel_24836/2162648250.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_df = data.groupby('company_name').apply(combine_rows).apply(pd.Series)


In [18]:
whole_data = []
for index, row in result_df.iterrows():
    concatenated_string = (
        f"company_name: {row['company_name']} \n "
        f"long_description: {row['long_description']} \n "
        f"batch: {row['batch']} \n "
        f"status: {row['status']} \n "
        f"tags: {row['tags']} \n "
        f"location: {row['location']} \n "
        f"country: {row['country']} \n "
        f"year_founded: {row['year_founded']} \n "
        f"num_founders: {row['num_founders']} \n "
        f"founders_names: {row['founders_names']} \n "
        f"team_size: {row['team_size']} \n "
        f"website: {row['website']} \n "
        f"cb_url: {row['cb_url']} \n "
        f"linkedin_url: {row['linkedin_url']} \n "
    )
    whole_data.append(concatenated_string)


In [19]:
print(whole_data[0])

company_name: 0pass 
 long_description: Compromised employee logins are the biggest cybersecurity risk to any business today - causing over 86% of security breaches.

0pass is an identity platform that allows organizations to enable the most secure login mechanisms available for every login across their entire company.

We replace traditional login methods like passwords with more convenient methods like Windows Hello, TouchID, FaceID, and Yubikeys. Generally these are “passwordless” methods.

Even if your employees fall victim to fraudulent emails - hackers will not be able to break into your company - period. We drive down account takeover and password resets by 96%.

Contact us to explore passwordless login for your business. 
Together, we'll be the start of 0 passwords everywhere. 
 90% of security breaches are caused by a simple employee mistake. 

Your employee gets a bad link, clicks an email, then types in their password, and maybe even their two-factor code - because it looks 

In [20]:
from langchain_core.documents import Document
documents = [Document(page_content=doc, metadata={'id': idx}) for idx, doc in enumerate(whole_data)]

In [21]:
def create_vector_store(docs, embeddings, store_name):
    persistent_directory = "./VectorStoresOld" + store_name

    print(f"\n--- Creating vector store {store_name} ---")

    Chroma.from_documents(docs, embeddings, persist_directory=persistent_directory)
    
    print(f"--- Finished creating vector store {store_name} ---")

In [22]:
huggingface_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

In [23]:
create_vector_store(documents, huggingface_embeddings, "/startup_data_vector_store")


--- Creating vector store /startup_data_vector_store ---
--- Finished creating vector store /startup_data_vector_store ---
